In [1]:
import flowshape as fs
import igl
import numpy as np
import meshplot as mp
import os
import pandas as pd
from src.utilities.fin_shape_utils import plot_mesh
from src.utilities.fin_class_def import FinData
from src.utilities.functions import path_leaf
import glob2 as glob
import meshplot as mp
import trimesh

## We're just gonna go for it!

### Load fin data

In [5]:
# get list of refined fin mesh objects
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
fin_mesh_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*smoothed_fin_mesh*")))

# load metadata
metadata_df = pd.read_csv(os.path.join(root, "metadata", "master_metadata.csv"))
metadata_df["experiment_date"] = metadata_df["experiment_date"].astype(str)
metadata_df.head()

# make savedir 
save_dir = os.path.join(root, "point_cloud_data", "SH_analysis", "")
y_subdir = os.path.join(save_dir, "Y_matrices")
if not os.path.isdir(y_subdir):
    os.makedirs(y_subdir)

### This is the norm utility in flowshape, but returns info I need for recovering original fin

In [6]:
from numpy.linalg import norm

def norm_verbose(verts):
    centroid = np.mean(verts, axis=0)
    verts -= centroid
    radii = norm(verts, axis=1)

    m = np.amax(radii)

    verts /= m
    return verts, centroid, m

### Iterate through fin meshes

In [7]:
# make col names to keep track of SH coefficients

def get_sh_colnames(max_degree):

    col_names = []
    
    for l in range(max_degree):
        
        i1 = l**2
        i2 = (l + 1) ** 2
    
        for m in range(i1, i2):
            col_name = f"sh_l{l:03}_m{m-i1:03}"
            col_names.append(col_name)
            
    return col_names


In [19]:
from tqdm import tqdm

max_degree = 30

# df_list = []
for file_ind, file_path in enumerate(tqdm(fin_mesh_list[107:])):
    # get colnames
    sh_colnames = get_sh_colnames(max_degree)
    
    # extract relevant metadata
    fname = os.path.basename(file_path)
    well_ind = fname.find("well")
    date_string = fname[:well_ind-1]
    well_num = int(fname[well_ind+4:well_ind+8])
    time_ind = fname.find("time")
    time_num = int(fname[time_ind+4:time_ind+8])
    
    # match this to a row in the metadata df
    date_ft = metadata_df["experiment_date"] == date_string
    well_ft = metadata_df["well_index"] == well_num
    time_ft = metadata_df["time_index"] == time_num
    
    meta_temp = metadata_df.loc[date_ft & well_ft & time_ft, :].reset_index(drop=True)

    cv = meta_temp.loc[0, "chem_i"]
    if isinstance(cv, str):
        cvs = cv.split("_")
        chem_id = cvs[0]
        chem_time = int(cvs[1])
    else:
        chem_id = "WT"
        chem_time = np.nan
    meta_temp.loc[0, "chem_id"] = chem_id
    meta_temp.loc[0, "chem_time"] = chem_time
    
    # make temp DF to store results
    
    # load mesh
    try:
        fin_mesh = trimesh.load(file_path)
        f, v = fin_mesh.faces.copy(), fin_mesh.vertices.copy()
        
        # normalize
        vn, mu, m = norm_verbose(v.copy())
        
        # perform SH decomposition
        weights, Y_mat, vs = fs.do_mapping(vn, f, l_max=max_degree)
        
        # store results
        meta_temp.loc[0, "scale"] = m.copy()
        meta_temp.loc[0, ["xc", "yc", "zc"]] = mu.copy()
        meta_temp.loc[0, sh_colnames] = weights.copy()
    
        df_list.append(meta_temp)
    
        # save Y
        yname = fname
        yname.replace("_smoothed_fin_mesh.obj", "y_mat.npy")
        np.save(os.path.join(y_subdir, yname), Y_mat)
    except:
        pass

SH_df = pd.concat(df_list, axis=0, ignore_index=True)
SH_df.to_csv(os.path.join(save_dir, "fin_sh_df.csv"), index=False)

  0%|          | 0/117 [00:00<?, ?it/s]

 72%|███████▏  | 84/117 [03:28<01:22,  2.51s/it]

 79%|███████▉  | 93/117 [03:51<01:01,  2.56s/it]

 85%|████████▍ | 99/117 [04:06<00:45,  2.52s/it]

 91%|█████████ | 106/117 [04:24<00:27,  2.52s/it]

 91%|█████████▏| 107/117 [04:27<00:25,  2.52s/it]

 92%|█████████▏| 108/117 [04:29<00:22,  2.52s/it]

100%|██████████| 117/117 [04:52<00:00,  2.50s/it]
